In [ ]:
#Importación de librerías
import io
import os
import re
from functools import reduce

import pandas as pd
from google.colab import files

In [ ]:
# Función auxiliar para detectar el separador del CSV
def detect_separator(text: str) -> str:
    """Detecta automáticamente el separador de campos del archivo."""
    first = text.splitlines()[0]
    if ";" in first:
        return ";"
    if "," in first:
        return ","
    return "\t"

# Convierte columnas H01–H24 a formato largo
def wide_to_long(df: pd.DataFrame, pollutant: str) -> pd.DataFrame:
    hour_cols = [c for c in df.columns if re.fullmatch(r"H\d{2}", c)]

    # Reorganiza las columnas horarias en filas
    long_df = df.melt(
        id_vars=["ANNO", "MES", "DIA", "ESTACION"],
        value_vars=hour_cols,
        var_name="hour",
        value_name=pollutant,
    )

    # Crea la columna de tiempo exacto con hora
    long_df["hour_num"] = long_df["hour"].str[1:].astype(int) - 1
    long_df["time"] = pd.to_datetime(
        dict(year=long_df["ANNO"], month=long_df["MES"], day=long_df["DIA"])
    )
    long_df["time"] += pd.to_timedelta(long_df["hour_num"], unit="h")

    long_df[pollutant] = pd.to_numeric(long_df[pollutant], errors="coerce")
    return long_df[["time", "ESTACION", pollutant]]

In [ ]:
# Proceso principal (para un año)
def process_year():
    print("Sube los seis archivos CSV correspondientes a un año (por ejemplo: CO_HH_2022.csv, …)")
    uploaded = files.upload()  # Diccionario: nombre → contenido en bytes

    tables: dict[str, list[pd.DataFrame]] = {}
    year: int | None = None

    for name, binary in uploaded.items():
        text = binary.decode("utf-8", errors="ignore")

        # Detectar contaminante y año a partir del nombre del archivo
        m = re.search(r"(CO|NO2|O3|PM10|PM25|SO2)_HH_(\d{4})", name.upper())
        if not m:
            print(f"Nombre de archivo no reconocido: {name}. Se omite.")
            continue
        pollutant, detected_year = m.group(1), int(m.group(2))
        year = year or detected_year

        # Leer el archivo CSV con el separador detectado
        df = pd.read_csv(io.StringIO(text), sep=detect_separator(text))

        # Filtrar Madrid (provincia 28, municipio 79)
        if {"PROVINCIA", "MUNICIPIO"}.issubset(df.columns):
            df = df[(df["PROVINCIA"] == 28) & (df["MUNICIPIO"] == 79)]
        else:
            print(f"Faltan columnas PROVINCIA o MUNICIPIO en {name}. Se omite.")
            continue

        tables.setdefault(pollutant, []).append(wide_to_long(df, pollutant))

    if not tables or year is None:
        raise RuntimeError("No se encontraron datos válidos entre los archivos subidos.")

    # Unir todos los contaminantes por fecha y estación
    pollutant_frames = [pd.concat(tbls, ignore_index=True) for tbls in tables.values()]
    merged = reduce(lambda left, right: pd.merge(left, right, on=["time", "ESTACION"], how="outer"), pollutant_frames)
    merged.sort_values(["time", "ESTACION"], inplace=True)

    # Guardar archivo final procesado
    out_name = f"contaminantes_{year}.csv"
    merged.to_csv(out_name, index=False)
    print(f"Archivo generado: {out_name}")
    files.download(out_name)

    # Eliminar archivos originales subidos para liberar espacio
    for raw_name in uploaded.keys():
        try:
            os.remove(raw_name)
        except Exception:
            pass


In [ ]:
# Ejecutar una vez por cada año
# Empezando por 2022:
process_year()

Sube los seis archivos CSV correspondientes a un año (por ejemplo: CO_HH_2022.csv, …)


Saving CO_HH_2022.csv to CO_HH_2022.csv
Saving NO2_HH_2022.csv to NO2_HH_2022.csv
Saving O3_HH_2022.csv to O3_HH_2022.csv
Saving PM10_HH_2022.csv to PM10_HH_2022.csv
Saving PM25_HH_2022.csv to PM25_HH_2022.csv
Saving SO2_HH_2022.csv to SO2_HH_2022.csv
Archivo generado: contaminantes_2022.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Seguimos con año 2023:
process_year()

Sube los seis archivos CSV correspondientes a un año (por ejemplo: CO_HH_2022.csv, …)


Saving CO_HH_2023.csv to CO_HH_2023.csv
Saving NO2_HH_2023.csv to NO2_HH_2023.csv
Saving O3_HH_2023.csv to O3_HH_2023.csv
Saving PM10_HH_2023.csv to PM10_HH_2023.csv
Saving PM25_HH_2023.csv to PM25_HH_2023.csv
Saving SO2_HH_2023.csv to SO2_HH_2023.csv
Archivo generado: contaminantes_2023.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>